In [1]:
import io
import os
import sys
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob
import copy
import time
from scipy import signal

In [2]:
sys.path.append(r'C:\Users\lesliec\code')

In [3]:
from tbd_eeg.tbd_eeg.data_analysis.eegutils import EEGexp, retEEGexp

In [4]:
%matplotlib notebook

### Functions

In [5]:
def parse_estim_protocol(stim_file):
    ## Read stim_protocol ##
    with open(stim_file) as file:
        estim_txt = file.read() # open estim file and read in text
    estim = estim_txt.split('channel:') # split and read txt file
    d = {}
    for item in estim:
        if 'value' in item:
            key = 'channel'+str(item[1])
            d[key]=item.split('\n')
    ## Read CHANNNEL 1 ##
    ch1_df = pd.read_csv(io.StringIO('\n'.join(d['channel1'][1:])), delim_whitespace=True)
    ## Get trials from stim_protocol ##
    all_stim_list = []
    eventid = 0
    for i in range(1, len(ch1_df)):
        if ch1_df.value.iloc[i-1] < 0 and ch1_df.value.iloc[i] > 0:
            all_stim_list.append([
                eventid, 'biphasic', ch1_df.value.iloc[i], (ch1_df.time.iloc[i-1] + ch1_df.time.iloc[i]) * 1e-6
            ])
            eventid += 1
    ch1_log = pd.DataFrame(all_stim_list, columns=['event_ID', 'stim_type', 'parameter', 'duration'])
    return ch1_log

def parse_opto_protocol(stim_file):
    ## Read stim_protocol ##
    with open(stim_file) as file:
        estim_txt = file.read() # open estim file and read in text
    estim = estim_txt.split('channel:') # split and read txt file
    d = {}
    for item in estim:
        if 'value' in item:
            key = 'channel'+str(item[1])
            d[key]=item.split('\n')
    ## Read CHANNNEL 4 ##
    ch4_df = pd.read_csv(io.StringIO('\n'.join(d['channel4'][1:])), delim_whitespace=True)
    durs = (ch4_df[ch4_df['value'] == 1]['time'].values[9:-9]) * 1E-6
    ch4_log = pd.DataFrame({'event_ID': np.arange(len(durs)), 'duration': durs,})
    return ch4_log

def parse_opto_protocol_dual1(stim_file):
    ## Read stim_protocol ##
    with open(stim_file) as file:
        estim_txt = file.read() # open estim file and read in text
    estim = estim_txt.split('channel:') # split and read txt file
    d = {}
    for item in estim:
        if 'value' in item:
            key = 'channel'+str(item[1])
            d[key]=item.split('\n')
    ## Read CHANNNEL 4 ##
    ch4_df = pd.read_csv(io.StringIO('\n'.join(d['channel4'][1:])), delim_whitespace=True)
    all_stim_list = []
    eventid = 0
    for i in range(1, len(ch4_df)):
        if ch4_df.value.iloc[i-1] == 1 and ch4_df.value.iloc[i] == 1:
            all_stim_list.append([eventid, (ch4_df.time.iloc[i-1] + ch4_df.time.iloc[i]) * 1e-6])
            eventid += 1
    ch4_log = pd.DataFrame(all_stim_list, columns=['event_ID', 'duration'])
    return ch4_log

## EEGexp class

In [6]:
# rec_folder = r'E:\eeg_pilot\mouse000000\estim_vis_test_2020-09-10\experiment1\recording1' # this is the test
rec_folder = r'X:\EEG_exp\mouse638703\urethane_estim_2022-10-14_12-25-20\experiment1\recording1'
exp = EEGexp(rec_folder, preprocess=False, make_stim_csv=False)

Experiment type: electrical stimulation


In [7]:
print(exp.mouse)
print(exp.experiment_type)
# print(exp.stim_instruction_files)
# print(exp.stimulus_log_file)

638703
electrical stimulation


In [8]:
sync_data = exp._load_sync_dataset()
print(sync_data.line_labels)

['barcodes', 'opto_sync', 'frames', 'behavior_sweep', 'photodiode', 'sweep', '', 'behavior_vsync', 'eyetracking', 'behavior', 'rotA', 'rotB', 'estim_sync', 'estim_sweep', '', '', '', 'opto_sweep', 'opto_trial', '', '', '', '', '', '', '', '', '', '', '', '', '']


In [9]:
# stimprot = {
#     0: ['opto', 5, 'NoEstim_Opto50ms_3.5-4.5sec_stimulation_protocol.txt'],
#     1: ['opto', 5, 'NoEstim_Opto100ms_3.5-4.5sec_stimulation_protocol.txt'],
#     2: ['estim', -1, 'Estim80uA_NoOpto_3.5-4.5sec_stimulation_protocol.txt'],
#     3: ['estim_opto', 5, 'Estim80uA_Opto50ms_-25ms_3.5-4.5sec_stimulation_protocol.txt'],
#     4: ['estim_opto', 5, 'Estim80uA_Opto100ms_+25ms_3.5-4.5sec_stimulation_protocol.txt'],
#     5: ['estim_opto', 5, 'Estim80uA_Opto100ms_+125ms_3.5-4.5sec_stimulation_protocol.txt'],
# }

stimprot = {
    0: ['estim', -1, 'Estim80uA_NoOpto_3.5-4.5sec_stimulation_protocol.txt'],
    1: ['estim', -1, 'Estim80uA_NoOpto_5-7sec_stimulation_protocol.txt'],
}

## EXAMPLES ##
#     0: ['opto', 1, 'NoEstim_Opto5ms_5-7sec_stimulation_protocol.txt'], for opto value is light intensity (mW)
#     5: ['opto', 2.5, 'NoEstim_Opto10ms_5-7sec_stimulation_protocol.txt'],
#     0: ['opto', 5, 'NoEstim_Opto50ms_5-7sec_stimulation_protocol.txt'],
#     4: ['estim', -1, 'Estim50uA_NoOpto_5-7sec_stimulation_protocol.txt'], for estim no value is needed
#     6: ['estim_opto', 5, 'Estim50uA_Opto500ms_-100ms_5-7sec_stimulation_protocol.txt'],
# for estim+opto value is light intensity (mW)

LED = '470nm'

## Create stim log

Load the sweep pulse, rising and falling

In [10]:
esweep_rising = sync_data.get_edges(keys=('estim_sweep',), kind='rising', units='seconds')
esweep_falling = sync_data.get_edges(keys=('estim_sweep',), kind='falling', units='seconds')
print('{:d} sweep rising edges and {:d} falling edges'.format(len(esweep_rising), len(esweep_falling)))

2 sweep rising edges and 2 falling edges


Load the estim sync pulses

In [12]:
esync_rising = sync_data.get_edges(keys=('estim_sync',), kind='rising', units='seconds')
esync_falling = sync_data.get_edges(keys=('estim_sync',), kind='falling', units='seconds')
print('{:d} estim-sync rising edges and {:d} falling edges'.format(len(esync_rising), len(esync_falling)))

240 estim-sync rising edges and 240 falling edges


Load the opto sync pulses

In [13]:
osync_rising = sync_data.get_edges(keys=('opto_sync',), kind='rising', units='seconds')
osync_falling = sync_data.get_edges(keys=('opto_sync',), kind='falling', units='seconds')
print('{:d} opto-sync rising edges and {:d} falling edges'.format(len(osync_rising), len(osync_falling)))

0 opto-sync rising edges and 0 falling edges


#### Need to determine if the sweep has estim, opto, or both stimuli

In [14]:
pulse_threshold = 20
for ii, (swstart, swend) in enumerate(zip(esweep_rising, esweep_falling)):
    ## Get pulses within the sweep ##
    esync_on_inds = np.nonzero((esync_rising >= swstart-3) & (esync_rising <= swend+2))[0]
    esync_off_inds = np.nonzero((esync_falling >= swstart-3) & (esync_falling <= swend+2))[0]
    osync_on_inds = np.nonzero((osync_rising >= swstart-3) & (osync_rising <= swend+2))[0]
    osync_off_inds = np.nonzero((osync_falling >= swstart-3) & (osync_falling <= swend+2))[0]
    
    if (len(esync_on_inds) > pulse_threshold) and (len(osync_on_inds) > pulse_threshold):
        print('Sweep {:d} has estim ({:d}) and opto ({:d})'.format(ii, len(esync_on_inds), len(osync_on_inds)))
    elif (len(esync_on_inds) > pulse_threshold) and (len(osync_on_inds) < pulse_threshold):
        print('Sweep {:d} has only estim ({:d}) / ({:d} opto)'.format(ii, len(esync_on_inds), len(osync_on_inds)))
    elif (len(osync_on_inds) > pulse_threshold) and (len(esync_on_inds) < pulse_threshold):
        print('Sweep {:d} has only opto ({:d}) / ({:d} estim)'.format(ii, len(osync_on_inds), len(esync_on_inds)))
    else:
        print('I dunno about sweep {:d}: estim={:d}, opto={:d}'.format(ii, len(esync_on_inds), len(osync_on_inds)))
    
    print('')

Sweep 0 has only estim (120) / (0 opto)

Sweep 1 has only estim (120) / (0 opto)



### Get logs for all sweeps

In [15]:
all_sweep_logs = []
for sweepi, sweep_data in stimprot.items():
    stimp_file = os.path.join(exp.data_folder, sweep_data[2])
    print('Sweep {:d}: {} ({})'.format(sweepi, sweep_data[0], sweep_data[2]))
    
    esync_on_inds = np.nonzero((esync_rising >= esweep_rising[sweepi]-3) & (esync_rising <= esweep_falling[sweepi]+2))[0]
    esync_off_inds = np.nonzero((esync_falling >= esweep_rising[sweepi]-3) & (esync_falling <= esweep_falling[sweepi]+2))[0]
    osync_on_inds = np.nonzero((osync_rising >= esweep_rising[sweepi]-3) & (osync_rising <= esweep_falling[sweepi]+2))[0]
    osync_off_inds = np.nonzero((osync_falling >= esweep_rising[sweepi]-3) & (osync_falling <= esweep_falling[sweepi]+2))[0]
    
    if sweep_data[0] == 'estim':
        ch1_templog = parse_estim_protocol(stimp_file)
        if len(ch1_templog) == len(esync_on_inds):
            temp_log = ch1_templog.copy()
            temp_log['event_type'] = [sweep_data[0]] * len(temp_log)
            temp_log['onset'] = esync_rising[esync_on_inds]
            temp_log['offset'] = esync_falling[esync_off_inds]
            temp_log['sweep'] = [sweepi] * len(temp_log)
            all_sweep_logs.append(temp_log)
        else:
            print(' Lengths of temp_stim_log and esync pulse edges do not match')
            continue

    elif (sweep_data[0] == 'opto') or (sweep_data[0] == 'opto_visual'):
        ch4_templog = parse_opto_protocol_dual1(stimp_file)
        if (len(osync_on_inds) == 138) and (len(osync_off_inds) == 138):
            osync_on_inds = osync_on_inds[9:-9]
            osync_off_inds = osync_off_inds[9:-9]
        elif (len(osync_on_inds) == 129) and (len(osync_off_inds) == 129):
            osync_on_inds = osync_on_inds[:-9]
            osync_off_inds = osync_off_inds[:-9]
        else:
            print(' Not enough opto sync pulses in this sweep')
            continue
            
        if len(ch4_templog) == len(osync_on_inds):
            temp_log = ch4_templog.copy()
            temp_log['event_type'] = [sweep_data[0]] * len(temp_log)
            temp_log['stim_type'] = [LED] * len(temp_log)
            temp_log['parameter'] = [sweep_data[1]] * len(temp_log)
            temp_log['onset'] = osync_rising[osync_on_inds]
            temp_log['offset'] = osync_falling[osync_off_inds]
            temp_log['sweep'] = [sweepi] * len(temp_log)
            all_sweep_logs.append(temp_log)
        else:
            print(' Lengths of temp_stim_log and osync pulse edges do not match')

    elif sweep_data[0] == 'estim_opto':
        ch1_templog = parse_estim_protocol(stimp_file)
        ch4_templog = parse_opto_protocol_dual1(stimp_file)
        if len(ch1_templog) == len(esync_on_inds):
            temp_log1 = ch1_templog.copy()
            temp_log1['event_type'] = [sweep_data[0]] * len(temp_log1)
            temp_log1['onset'] = esync_rising[esync_on_inds]
            temp_log1['offset'] = esync_falling[esync_off_inds]
            temp_log1['sweep'] = [sweepi] * len(temp_log1)
        else:
            print(' Lengths of temp_stim_log and esync pulse edges do not match')
            continue

        if (len(osync_on_inds) == 138) and (len(osync_off_inds) == 138):
            osync_on_inds = osync_on_inds[9:-9]
            osync_off_inds = osync_off_inds[9:-9]
        elif (len(osync_on_inds) == 129) and (len(osync_off_inds) == 129):
            osync_on_inds = osync_on_inds[:-9]
            osync_off_inds = osync_off_inds[:-9]
        else:
            print(' Not enough opto sync pulses in this sweep')
            continue
            
        if len(ch4_templog) == len(osync_on_inds):
            temp_log2 = ch4_templog.copy()
            temp_log2['event_type'] = [sweep_data[0]] * len(temp_log2)
            temp_log2['stim_type'] = [LED] * len(temp_log2)
            temp_log2['parameter'] = [sweep_data[1]] * len(temp_log2)
            temp_log2['onset'] = osync_rising[osync_on_inds]
            temp_log2['offset'] = osync_falling[osync_off_inds]
            temp_log2['sweep'] = [sweepi] * len(temp_log2)
        else:
            print(' Lengths of temp_stim_log and osync pulse edges do not match')

        if len(temp_log1) == len(temp_log2):
            temp_log = pd.concat((temp_log1, temp_log2), axis=0, sort=False).sort_values(by='onset', axis=0, ignore_index=True)
            all_sweep_logs.append(temp_log)

    else:
        print(' Event type not understood.')
        continue
        
    print('')

Sweep 0: estim (Estim80uA_NoOpto_3.5-4.5sec_stimulation_protocol.txt)

Sweep 1: estim (Estim80uA_NoOpto_5-7sec_stimulation_protocol.txt)



In [16]:
len(all_sweep_logs)

2

In [17]:
all_stim_log = pd.concat(all_sweep_logs, axis=0, sort=False).sort_values(by='onset', axis=0, ignore_index=True)
all_stim_log = all_stim_log[['event_type', 'event_ID', 'stim_type', 'parameter', 'onset', 'offset', 'duration', 'sweep']]

In [18]:
all_stim_log.tail()

,event_type,event_ID,stim_type,parameter,onset,offset,duration,sweep
235,estim,115,biphasic,80,3442.61320,3442.61360,0.0004,1
236,estim,116,biphasic,80,3448.76606,3448.76646,0.0004,1
237,estim,117,biphasic,80,3455.13322,3455.13362,0.0004,1
238,estim,118,biphasic,80,3461.22683,3461.22723,0.0004,1
239,estim,119,biphasic,80,3467.07873,3467.07913,0.0004,1


## Ready to save?

In [19]:
all_stim_log.to_csv(exp.stimulus_log_file, index=False)

## Testing